# 01_data_prep_imputation


In [ ]:
import pandas as pd

data = pd.read_excel('FullData.xlsx')

data.to_csv('FullData.csv', index=False)

df = pd.read_csv('FullData.csv')

Databackup = df

imputed_df=pd.read_csv('FullDataImputed1.csv')

cat_cols = ['RAILROAD', 'STATE ', 'VISIBLTY', 'WEATHER', 'TYPE', 'TYPEQ', 'TRKCLAS', 'AMPM', 'CAUSE', 'ACCTRK', 'TYPTRK']



In [ ]:
for col in cat_cols:
    Databackup[col] = Databackup[col].apply(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
import numpy as np

df = Databackup
df = df.drop(['RR3','CAUSE2','ACCTRKCL','ENGRS','FIREMEN','CONDUCTR','BRAKEMEN'], axis=1)

cat_cols = ['RAILROAD',  'STATE ', 'VISIBLTY','WEATHER' , 'TYPE','TYPEQ', 'TRKCLAS', 'AMPM', 'CAUSE','ACCTRK','TYPTRK']

is_null = df.isna() | df.eq('#NA') | df.eq('')

matrix = np.column_stack(np.where(is_null))

column_names = list(df.columns)

null_cells = [(row, column_names[col], df.iloc[row, col]) for row, col in matrix]

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

null_cells_count2 = null_cells_df2.groupby('Column')['Value'].count()

for col in cat_cols:
    df[col] = df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

print(null_cells_df2['Column'].value_counts())
print(null_cells_df2.head())
num_rows = null_cells_df2.shape[0]
print(f'The null_cells_df has {num_rows} rows.')


In [ ]:
print(imputed_df.columns)
print(df.columns)


In [ ]:
row_dict = df.iloc[1].to_dict()
for key, value in row_dict.items():
    print(key, ':', value, end=', ')
print(imputed_df.iloc[32844])


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
import scipy.stats as st

dataCor = df.dropna()

cat_cols = ['RAILROAD',  'STATE ', 'VISIBLTY','WEATHER' , 'TYPE','TYPEQ', 'TRKCLAS', 'AMPM', 'CAUSE','ACCTRK','TYPTRK']
dataCor[cat_cols] = dataCor[cat_cols].astype('category')

corr_matrix = pd.DataFrame(index=dataCor.columns, columns=dataCor.columns, dtype=float)
for i, col1 in enumerate(dataCor.columns):
    for j, col2 in enumerate(dataCor.columns):
        if i <= j:
            if dataCor[col1].dtype.name == 'category' and dataCor[col2].dtype.name == 'category':
                cross_tab = pd.crosstab(dataCor[col1], dataCor[col2])
                if cross_tab.shape[0] > 1 and cross_tab.shape[1] > 1:
                    chi2, p_val, dof, expected = chi2_contingency(cross_tab)
                    n = cross_tab.sum().sum()
                    phi2 = chi2/n
                    r,k = cross_tab.shape
                    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
                    rcorr = r - ((r-1)**2)/(n-1)
                    kcorr = k - ((k-1)**2)/(n-1)
                    corr = np.sqrt(phi2corr / min((kcorr-1),(rcorr-1)))
                    corr_matrix.loc[col1, col2] = corr
                    corr_matrix.loc[col2, col1] = corr
            elif dataCor[col1].dtype.name != 'category' and dataCor[col2].dtype.name != 'category':
                corr, p_val = pearsonr(dataCor[col1], dataCor[col2])
                corr_matrix.loc[col1, col2] = corr
                corr_matrix.loc[col2, col1] = corr

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(corr_matrix)



In [ ]:
corr_matrix.to_csv("correlation_matrix.csv", index=True, header=True)


In [ ]:
print(df['STATE '].unique())
print(imputed_df['STATE '].unique())


In [ ]:
print(df.loc[67024, 'TYPEQ'])


In [ ]:
df = Databackup
df = df.drop(['RR3','CAUSE2','ACCTRKCL','ENGRS','FIREMEN','CONDUCTR','BRAKEMEN'], axis=1)
print(df['STATE '].unique())


In [ ]:
sums_dict = {}
dfimpute=df
class_mapping = {'x': -1, 'o': -2}

dfimpute['TRKCLAS'] = dfimpute['TRKCLAS'].replace({'1.0':'1','4.0':'4','2.0': '2', '3.0': '3'}).replace(class_mapping)
dfimpute['TRKCLAS'] = dfimpute['TRKCLAS'].apply(lambda x: int(x) if pd.notnull(x) else x)

for index, row in null_cells_df.iterrows():
    sums_dict = {}
    missing_col = row['Column']
    missing_row = row['Row']
    if missing_col in cat_cols:
        groupby_cols = [col for col in dfimpute.columns if col != missing_col]

        for col in groupby_cols:
            filter_val = dfimpute.loc[missing_row, col]

            groupby_df = dfimpute[dfimpute[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()
                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        category = sums_df['count_sum'].idxmax()
        dfimpute.loc[missing_row, missing_col] = category


In [ ]:
columns_to_compare = imputed_df.columns

comparison_df = df[columns_to_compare].compare(imputed_df[columns_to_compare], keep_equal=True)
print(comparison_df)

similar_records = comparison_df[comparison_df.iloc[:, 1].isnull()]
similar_indices = similar_records.index

num_similar_rows = len(similar_records)
print(f"Number of similar rows: {num_similar_rows}")


In [ ]:

comparison_df = df.compare(imputed_df, keep_equal=True)

similar_records = comparison_df[comparison_df.iloc[:, 1].isnull()]
similar_indices = similar_records.index

num_similar_rows = len(similar_records)
print(f"Number of similar rows: {num_similar_rows}")


In [ ]:
print(df.columns)


In [ ]:

col_to_impute =['VISIBLTY','WEATHER']
sums_dict = {}
dfimputed2=df

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    print(f"missingcol:{missing_col}")
    missing_row = row['Row']
    print(f"missingrow:{missing_row}")
    if missing_col in cat_cols:
        groupby_cols = ['AMPM', 'STATE ', 'MONTH']

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)
            print(groupby_df)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        print(f"Final result:{sums_df}")
        category = sums_df['count_sum'].idxmax()
        print(category)
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
df = Databackup
if (df['VISIBLTY'].isna() | df['VISIBLTY'].eq('#NA') | df['VISIBLTY'].eq('')).any():
    print("There are missing or empty values in the 'VISIBLTY' column")

if (df['WEATHER'].isna() | df['WEATHER'].eq('#NA') | df['WEATHER'].eq('')).any():
    print("There are missing or empty values in the 'WEATHER' column")


In [ ]:
if (dfimputed2['VISIBLTY'].isna() | dfimputed2['VISIBLTY'].eq('#NA') | dfimputed2['VISIBLTY'].eq('')).any():
    print("There are missing or empty values in the 'VISIBLTY' column")

if (dfimputed2['WEATHER'].isna() | dfimputed2['WEATHER'].eq('#NA') | dfimputed2['WEATHER'].eq('')).any():
    print("There are missing or empty values in the 'WEATHER' column")


In [ ]:

col_to_impute =['TYPEQ']
sums_dict = {}

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    missing_row = row['Row']
    if missing_col in cat_cols:
        groupby_cols = ['RAILROAD', 'STATE ']

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        category = sums_df['count_sum'].idxmax()
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
if (dfimputed2['TYPEQ'].isna() | dfimputed2['TYPEQ'].eq('#NA') | dfimputed2['TYPEQ'].eq('')).any():
    print("There are missing or empty values in the 'typeq' column")


In [ ]:

col_to_impute =['TYPE']
sums_dict = {}

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    missing_row = row['Row']
    if missing_col in cat_cols:
        groupby_cols = ['CAUSE']

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        category = sums_df['count_sum'].idxmax()
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
if (dfimputed2['TYPE'].isna() | dfimputed2['TYPE'].eq('#NA') | dfimputed2['TYPE'].eq('')).any():
    print("There are missing or empty values in the 'typeq' column")


In [ ]:

col_to_impute =['STATE ']
sums_dict = {}

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    print(f"missingcol:{missing_col}")
    missing_row = row['Row']
    print(f"missingrow:{missing_row}")
    if missing_col in cat_cols:
        groupby_cols = ['RAILROAD']

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)
            print(groupby_df)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        print(f"Final result:{sums_df}")
        category = sums_df['count_sum'].idxmax()
        print(category)
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
if (dfimputed2['STATE '].isna() | dfimputed2['STATE '].eq('#NA') | dfimputed2['STATE '].eq('')).any():
    print("There are missing or empty values in the 'STATE ' column")


In [ ]:

col_to_impute =['TYPTRK']
sums_dict = {}

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    print(f"missingcol:{missing_col}")
    missing_row = row['Row']
    print(f"missingrow:{missing_row}")
    if missing_col in cat_cols:
        groupby_cols = ['TRNSPD','HIGHSPD']

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)
            print(groupby_df)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        category = sums_df['count_sum'].idxmax()
        print(category)
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
if (dfimputed2['TYPTRK'].isna() | dfimputed2['TYPTRK'].eq('#NA') | dfimputed2['TYPTRK'].eq('')).any():
    print("There are missing or empty values in the 'TYPTRK' column")


In [ ]:

col_to_impute =['TRKCLAS']
sums_dict = {}

class_mapping = {'x': -1, 'o': -2}
dfimputed2['TRKCLAS'] = dfimputed2['TRKCLAS'].replace({'1.0':'1','4.0':'4','2.0': '2', '3.0': '3'}).replace(class_mapping)
dfimputed2['TRKCLAS'] = dfimputed2['TRKCLAS'].apply(lambda x: int(x) if pd.notnull(x) else x)

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    print(f"missingcol:{missing_col}")
    missing_row = row['Row']
    print(f"missingrow:{missing_row}")
    if missing_col in cat_cols:
        if pd.isnull(dfimputed2.loc[missing_row, 'TRKCLAS']) and pd.isnull(dfimputed2.loc[missing_row, 'ACCTRK']):
            groupby_cols = ['CAUSE']
            print(groupby_cols)
        else:
            groupby_cols = ['ACCTRK']
            print(groupby_cols)

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)
            print(groupby_df)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        category = sums_df['count_sum'].idxmax()
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
if (dfimputed2['TRKCLAS'].isna() | dfimputed2['TRKCLAS'].eq('#NA') | dfimputed2['TRKCLAS'].eq('')).any():
    print("There are missing or empty values in the 'TRKCLAS' column")


In [ ]:

col_to_impute =['ACCTRK']
sums_dict = {}

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

for index, row in null_cells_df2[null_cells_df2['Column'] .isin(col_to_impute)].iterrows():
    sums_dict = {}
    missing_col = row['Column']
    missing_row = row['Row']

    if missing_col in cat_cols:
        if pd.isnull(dfimputed2.loc[missing_row, 'TRKCLAS']) and pd.isnull(dfimputed2.loc[missing_row, 'ACCTRK']):
            groupby_cols = ['CAUSE']
            print(groupby_cols)
        else:
            groupby_cols = ['TRKCLAS']
            print(groupby_cols)

        for col in groupby_cols:
            filter_val = dfimputed2.loc[missing_row, col]

            groupby_df = dfimputed2[dfimputed2[col] == filter_val].groupby([col, missing_col]).size().reset_index(name='count')

            groupby_df = groupby_df.sort_values('count', ascending=False)

            for category in groupby_df[missing_col].unique():
                if category in sums_dict:
                    sums_dict[category] += groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

                else:
                    sums_dict[category] = groupby_df.loc[groupby_df[missing_col] == category, 'count'].sum()

        sums_df = pd.DataFrame.from_dict(sums_dict, orient='index', columns=['count_sum'])

        category = sums_df['count_sum'].idxmax()
        dfimputed2.loc[missing_row, missing_col] = category



In [ ]:
if (dfimputed2['ACCTRK'].isna() | dfimputed2['ACCTRK'].eq('#NA') | dfimputed2['ACCTRK'].eq('')).any():
    print("There are missing or empty values in the 'TRKCLAS' column")


In [ ]:
cat_cols = ['RAILROAD',  'STATE ', 'VISIBLTY','WEATHER' , 'TYPE','TYPEQ', 'TRKCLAS', 'AMPM', 'CAUSE','ACCTRK','TYPTRK']

is_null = dfimputed2.isna() | dfimputed2.eq('#NA') | dfimputed2.eq('')

matrix = np.column_stack(np.where(is_null))

column_names = list(dfimputed2.columns)

null_cells = [(row, column_names[col], dfimputed2.iloc[row, col]) for row, col in matrix]

null_cells_df2 = pd.DataFrame(null_cells, columns=['Row', 'Column', 'Value'])

null_cells_count2 = null_cells_df2.groupby('Column')['Value'].count()

for col in cat_cols:
    dfimputed2[col] = dfimputed2[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

print(null_cells_df2['Column'].value_counts())
print(null_cells_df2.head())
num_rows = null_cells_df2.shape[0]
print(f'The null_cells_df has {num_rows} rows.')


In [ ]:
dfimputed2.to_csv('dfimputed2.csv', index=False)


In [ ]:
cat_cols = ['RAILROAD', 'STATE ', 'VISIBLTY', 'WEATHER', 'TYPE', 'TYPEQ', 'TRKCLAS', 'AMPM', 'CAUSE', 'ACCTRK', 'TYPTRK']

df1 = pd.read_csv('dfimputed2.csv')
df2 = pd.read_csv('FullDataImputed1.csv')

for col in cat_cols:
    df2[col] = df2[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

diff = df1 != df2

num_diff = diff.sum().sum()

print(f"Number of differences: {num_diff}")

rows, cols = np.where(diff)

for row, col in zip(rows, cols):

    if pd.isna(df.iloc[row, df.columns.get_loc(df1.columns[col])]) or df.iloc[row, df.columns.get_loc(df1.columns[col])] == "#NA" or pd.isnull(df.iloc[row, df.columns.get_loc(df1.columns[col])]):
        continue
    else:
        print(f"Row: {row}, Column: {df1.columns[col]} ")
        value = df.loc[row, df1.columns[col]]
        print(f"df value of this cell: {value}")


In [ ]:
df = df.drop(['RR3','CAUSE2','ACCTRKCL','ENGRS','FIREMEN','CONDUCTR','BRAKEMEN'], axis=1)
print(df1.columns)
print(df2.columns)
print(df.columns)


In [ ]:
null_cells_count = df.isna().sum().sum()
na_cells_count = (df == "#NA").sum().sum()
print(f"total number of na cells is : {null_cells_count}")
print(f"total number of #NA is: {na_cells_count}")
